# Direct Distance
In this approach, I make the assumption that the distance between two shops is independent of the road network.

## Generate Random Warehouses

In [1]:
import copy
import os
import pickle
import random

import pulp as pl
from geopy.geocoders import Nominatim
from haversine import Unit, haversine_vector

In [2]:
if not os.path.exists("warehouses.pkl"):
    print("Generating warehouses info...")

    warehouses = []
    for i in range(1, 101):
        name = f"Warehouse {i}"
        lat = random.uniform(
            10.668799, 10.878041
        )  # Latitude range of Ho Chi Minh City
        long = random.uniform(
            106.451840, 106.813580
        )  # Longitude range of Ho Chi Minh City
        description = None
        warehouses.append(
            {
                "index": i,
                "name": name,
                "lat": lat,
                "long": long,
                "description": "",
            }
        )

    geolocator = Nominatim(user_agent="bk-imp")

    for i in range(len(warehouses)):
        lat, long = warehouses[i]["lat"], warehouses[i]["long"]
        location = geolocator.reverse(f"{lat}, {long}")
        address = location.address
        warehouses[i]["description"] = address

    with open("warehouses.pkl", "wb") as f:
        # Shuffle warehouses to generate random order
        random.shuffle(warehouses)
        pickle.dump(warehouses, f)
else:
    with open("warehouses.pkl", "rb") as f:
        warehouses = pickle.load(f)

In [3]:
with open("stores.pkl", "rb") as f:
    stores = pickle.load(f)

## Calculate Distance Matrix

In [4]:
warehouses_stores_dinstance_matrix = haversine_vector(
    [(store["lat"], store["long"]) for store in stores],
    [(warehouse["lat"], warehouse["long"]) for warehouse in warehouses],
    Unit.KILOMETERS,
    comb=True,
)

In [5]:
warehouses_stores_dinstance_matrix.shape

(100, 1000)

## Linear Programming

In [6]:
# Set up the problem
problem = pl.LpProblem("Warehouse_Optimization", pl.LpMinimize)

### Define the environment

In [7]:
M = 100  # Number of warehouses
N = 1000  # Number of stores
warehouse_ids = range(M)
w_cost = random.sample(range(100, 201), M)  # Operational cost of warehouses
w_capacity = random.sample(
    range(13000, 15001), M
)  # Operational cost of warehouses
store_ids = range(N)
demands = random.sample(range(10, 1101), N)  # Daily demand of stores
d_cost = copy.copy(
    warehouses_stores_dinstance_matrix
)  # Delivery cost between warehouses vs stores

In [8]:
assert sum(w_capacity) > sum(
    demands
), "All warehouses' capacity is less than stores' demands"

### Define the decision variables

In [10]:
wr = pl.LpVariable.dicts(
    "warehouse_store_connection", (warehouse_ids, store_ids), cat="Binary"
)

### Define the objective function

In [11]:
total_cost = pl.lpSum(
    w_cost[w] * wr[w][s] + d_cost[w][s] * wr[w][s]
    for w in warehouse_ids
    for s in store_ids
)
problem += total_cost

### Define the constraints

In [12]:
# Each store must be assigned to one warehouse
for i in store_ids:
    problem += pl.lpSum(wr[j][i] for j in warehouse_ids) == 1

In [13]:
# Total demand of assigned stores for a warehouse must be smaller than its capacity
for w in warehouse_ids:
    problem += (
        pl.lpSum(demands[s] * wr[w][s] for s in store_ids) <= w_capacity[w]
    )

## Solve the problem!!!

### COIN-OR Branch and Cut solver (CBC)
https://coin-or.github.io/Cbc/intro.html#:~:text=The%20COIN%2DOR%20Branch%20and,executable%20version%20is%20also%20available.

In [14]:
problem.solve(pl.PULP_CBC_CMD(timeLimit=120, threads=6, msg=0))

1

In [15]:
# Print the optimal solution
print("Optimal Solution:")
for w in warehouse_ids:
    assigned_stores = []
    for s in store_ids:
        if pl.value(wr[w][s]) == 1:
            assigned_stores.append(s)
    print(f"Warehouse {w} serves these stores:", assigned_stores)

# Print the total cost
print("Total Cost: ", pl.value(problem.objective))

Optimal Solution:
Warehouse 0 serves these stores: [617]
Warehouse 1 serves these stores: [164, 505]
Warehouse 2 serves these stores: []
Warehouse 3 serves these stores: [579]
Warehouse 4 serves these stores: [480]
Warehouse 5 serves these stores: [417]
Warehouse 6 serves these stores: [14, 120, 151, 303, 352, 467, 492, 494, 517, 568, 582, 883, 914, 941, 984, 987]
Warehouse 7 serves these stores: [306, 391, 561, 746]
Warehouse 8 serves these stores: [6, 91, 232, 424, 437, 456, 626, 729, 816]
Warehouse 9 serves these stores: []
Warehouse 10 serves these stores: [153]
Warehouse 11 serves these stores: [308]
Warehouse 12 serves these stores: [0, 13, 20, 79, 137, 143, 181, 185, 191, 213, 269, 328, 338, 342, 422, 489, 490, 511, 514, 542, 577, 586, 619, 635, 664, 665, 739, 740, 850, 854, 869, 899, 919, 956, 958, 961, 971, 982, 996]
Warehouse 13 serves these stores: []
Warehouse 14 serves these stores: []
Warehouse 15 serves these stores: [46, 127, 170, 326, 368, 513, 537, 553, 554, 633, 661,

### CPLEX Solver
Get it free from here https://www.ibm.com/academic/topic/data-science

or https://storage.googleapis.com/thaitang-sharing/cplex_studio2211.linux_x86_64.bin

In [16]:
os.environ["CPLEX_HOME"] = "/opt/ibm/ILOG/CPLEX_Studio2211/cplex"
os.environ["CPO_HOME"] = "/opt/ibm/ILOG/CPLEX_Studio2211/cpoptimizer"
os.environ["PATH"] += (
    ":"
    + os.environ["CPLEX_HOME"]
    + "/bin/x86-64_linux:"
    + os.environ["CPO_HOME"]
    + "/bin/x86-64_linux"
)
os.environ["LD_LIBRARY_PATH"] += (
    ":"
    + os.environ["CPLEX_HOME"]
    + "/bin/x86-64_linux:"
    + os.environ["CPO_HOME"]
    + "/bin/x86-64_linux"
)
os.environ[
    "PYTHONPATH"
] = "/opt/ibm/ILOG/CPLEX_Studio2211/cplex/python/3.10/x86-64_linux"

In [17]:
problem.solve(pl.CPLEX_CMD(timeLimit=120, threads=6, msg=0))

1

In [18]:
# Print the optimal solution
print("Optimal Solution:")
for w in warehouse_ids:
    assigned_stores = []
    for s in store_ids:
        if pl.value(wr[w][s]) == 1:
            assigned_stores.append(s)
    print(f"Warehouse {w} serves these stores:", assigned_stores)

# Print the total cost
print("Total Cost: ", pl.value(problem.objective))

Optimal Solution:
Warehouse 0 serves these stores: []
Warehouse 1 serves these stores: []
Warehouse 2 serves these stores: []
Warehouse 3 serves these stores: []
Warehouse 4 serves these stores: []
Warehouse 5 serves these stores: []
Warehouse 6 serves these stores: [461, 492, 529, 531, 606, 634, 648, 787, 872, 877, 914, 984, 987, 989]
Warehouse 7 serves these stores: []
Warehouse 8 serves these stores: [281, 870]
Warehouse 9 serves these stores: []
Warehouse 10 serves these stores: []
Warehouse 11 serves these stores: []
Warehouse 12 serves these stores: [0, 13, 20, 70, 77, 86, 95, 137, 143, 145, 181, 191, 213, 269, 338, 367, 382, 407, 427, 457, 490, 494, 496, 511, 514, 542, 554, 577, 619, 620, 635, 641, 664, 724, 731, 739, 740, 786, 823, 828, 846, 850, 854, 899, 902, 919, 930, 956, 958, 971, 996]
Warehouse 13 serves these stores: []
Warehouse 14 serves these stores: []
Warehouse 15 serves these stores: [46, 98, 116, 243, 326, 352, 439, 545, 560, 603, 661, 698, 699, 728, 794, 853, 918

# Actual Distance
Actual distance is calculated by calling the API of OpenStreetMap.

Note that the Openrouteservice API has a limit of 20 requests per minute for Isochrones.

In [19]:
# Update the actual distance from warehouse to assigned store
for w in warehouse_ids:
    for s in store_ids:
        if pl.value(wr[w][s]) == 1:
            print(d_cost[w][s])
            print(w, s)
            print(warehouses[w])
            break

3.2165365586749255
6 461
{'index': 10, 'name': 'Warehouse 10', 'lat': 10.798339649744692, 'long': 106.61684724656072, 'description': 'Đường Tân Kỳ Tân Quý, Phường Tân Quý, Quận Tân Phú, Thành phố Hồ Chí Minh, Huyện Tân Phú Đông, 72011, Việt Nam'}
1.0435856294139811
8 281
{'index': 49, 'name': 'Warehouse 49', 'lat': 10.838871561912113, 'long': 106.78627864512752, 'description': 'Phường Tăng Nhơn Phú A, Thành phố Thủ Đức, Thành phố Hồ Chí Minh, 00848, Việt Nam'}
1.5826665279749965
12 0
{'index': 55, 'name': 'Warehouse 55', 'lat': 10.828291062546038, 'long': 106.62478789040411, 'description': 'Trường Chinh, Phường Tân Hưng Thuận, Quận 12, Thành phố Hồ Chí Minh, 71509, Việt Nam'}
6.391032692352484
15 46
{'index': 93, 'name': 'Warehouse 93', 'lat': 10.786496371091065, 'long': 106.45588968960037, 'description': 'Xã Hựu Thạnh, Huyện Đức Hòa, Tỉnh Long An, Việt Nam'}
2.0621434252896207
16 2
{'index': 51, 'name': 'Warehouse 51', 'lat': 10.795329803956442, 'long': 106.71803675467702, 'descriptio

In [29]:
import requests
from dotenv import dotenv_values

base_url = "https://api.openrouteservice.org/v2/directions/driving-car"
env_vars = dotenv_values(".env")
api_key = env_vars["API_KEY"]  # Retrieve API key from .env file


def calculate_distance(lat1: float, lon1: float, lat2: float, lon2: float):
    params = {
        "api_key": api_key,
        "start": f"{lon1},{lat1}",
        "end": f"{lon2},{lat2}",
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if "features" in data:
        distance = data["features"][0]["properties"]["summary"]["distance"]
        return distance / 1000  # km
    else:
        return None


# Example usage
lat1 = 10.755094111557883
lon1 = 106.61290703156426
lat2 = 10.859352123859617
lon2 = 106.55368024272815

distance = calculate_distance(lat1, lon1, lat2, lon2)
print(f"The distance between the two points is {distance} kilometers.")

The distance between the two points is 18.5932 kilometers.
